In [1]:
import pickle
import logging
import pandas as pd
import spotipy

from spotipy.oauth2 import SpotifyClientCredentials
from datetime import datetime

logger = logging.getLogger()


In [2]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [3]:
spotify = spotipy.Spotify(client_credentials_manager=SpotifyClientCredentials())



Pour avoir les playlist de l'utilisateur loggé:  
 current_user_playlists(limit=50, offset=0) 
On peut aussi avoir les playlists depuis un id entré manuellement   
Ou en passant par les id des utilisateur : on récuperere les playlists via user ID   
(user) user_playlist duquel on extrait l'id de la playlist dont on recupere les musiques avec  playlist(playlist_id, fields=None, market=None, additional_types=('track', ))   
puis  playlist_tracks(playlist_id, fields=None, limit=100, offset=0, market=None, additional_types=('track', )) ??

VERIFIER QUON PREND BIEN QUE DES TRACKS ET PAS DES PODCASTS OU AUTRE

In [4]:
playlists2 = spotify.playlist(playlist_id='2seEyHNHF6LiVzLRQjSwHn')


In [5]:
playlists = spotify.playlist_tracks(playlist_id='2seEyHNHF6LiVzLRQjSwHn')


In [6]:
playlists2['tracks']['items'] == playlists['items']

True

Similaire à celle dans import-nb: Factoriser ? 

In [7]:
list_playlist_id = ['2seEyHNHF6LiVzLRQjSwHn']

In [8]:
tracks_data_list = []

start = datetime.now()
print("start = ", start)
try:
    for playlist_id in list_playlist_id:
        track_results = spotify.playlist_tracks(playlist_id=playlist_id)
        while track_results: 
            for track in track_results['items']:
                single_track_dict = {                                       
                    'track_name': track['track']['name'],
                    'artist_name': track['track']['artists'][0]['name'],
                    'track_id': track['track']['id'] ,
                    'popularity': track['track']['popularity']
                }
                tracks_data_list.append(single_track_dict)
            track_results = spotify.next(track_results)
        print(f"Progression of playlists: {list_playlist_id.index(playlist_id)+1}/{len(list_playlist_id)}")
except Exception as e:
    logger.error(e)
end = datetime.now()
print("finish = ", end)
print("duration = ", (end-start).total_seconds())

start =  2022-08-27 16:23:35.487397
Progression of playlists: 1/1
finish =  2022-08-27 16:23:36.934286
duration =  1.446889


In [9]:
len(tracks_data_list)

661

In [10]:
df = pd.DataFrame(tracks_data_list)

In [11]:
df.duplicated().sum()

0

In [12]:
df.drop_duplicates(inplace=True)

In [13]:
df.shape

(661, 4)

In [14]:
def extract_audio_feature(ids_list):
    list_features = []
    try:
        list_features = spotify.audio_features(ids_list)
    except Exception as e:
        print(e)
        logger.info(e)
    return list_features

In [15]:
tracks_features_list = []

total_tracks = df.shape[0]

start = datetime.now()
print("start = ", start)
for tracks_nbr in range(0, total_tracks, 100):
    print(f"Progression of tracks: {tracks_nbr}/{total_tracks}")
    tracks_features_list += extract_audio_feature(','.join(df.track_id[tracks_nbr:tracks_nbr+100]))
remaining_tracks = total_tracks%100
if remaining_tracks:
    tracks_features_list += extract_audio_feature(','.join(df.track_id[-remaining_tracks:]))
end = datetime.now()
print("finish = ", end)
print("duration = ", (end-start).total_seconds())

start =  2022-08-27 16:23:37.004644
Progression of tracks: 0/661
Progression of tracks: 100/661
Progression of tracks: 200/661
Progression of tracks: 300/661
Progression of tracks: 400/661
Progression of tracks: 500/661
Progression of tracks: 600/661
finish =  2022-08-27 16:23:38.265878
duration =  1.261234


In [16]:
len(tracks_features_list)

722

In [17]:
tracks_features_list = [track for track in tracks_features_list if isinstance(track, dict)]

In [18]:
len(tracks_features_list)

722

In [19]:
df2 = pd.DataFrame(tracks_features_list)

In [20]:
df2.shape

(722, 18)

In [21]:
print(df2.duplicated().sum())
df2.drop_duplicates(inplace=True)
df2.shape

61


(661, 18)

In [22]:
final_df = df.set_index('track_id').join(df2.set_index('id'), how='inner')

In [23]:
final_df.shape

(661, 20)

In [24]:
final_df.sample(10)

,track_name,artist_name,popularity,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,type,uri,track_href,analysis_url,duration_ms,time_signature
35mL8keNDi2VLHrFXZCgJR,Shameful Company,Rainbow Kitten Surprise,48,0.699,0.458,0,-12.357,1,0.1060,0.3720,0.000001,0.1110,0.269,129.991,audio_features,spotify:track:35mL8keNDi2VLHrFXZCgJR,https://api.spotify.com/v1/tracks/35mL8keNDi2V...,https://api.spotify.com/v1/audio-analysis/35mL...,221546,4
25SJaCCPiosugnlY53VhRG,Freaking Out the Neighborhood,Mac DeMarco,1,0.565,0.832,8,-5.751,0,0.1080,0.0639,0.029100,0.3860,0.847,143.302,audio_features,spotify:track:25SJaCCPiosugnlY53VhRG,https://api.spotify.com/v1/tracks/25SJaCCPiosu...,https://api.spotify.com/v1/audio-analysis/25SJ...,173889,4
33e9HtOfO1ye1h8xQ7Foc9,The Cave,Mumford & Sons,0,0.592,0.499,4,-9.463,1,0.0374,0.0536,0.000044,0.1130,0.345,141.938,audio_features,spotify:track:33e9HtOfO1ye1h8xQ7Foc9,https://api.spotify.com/v1/tracks/33e9HtOfO1ye...,https://api.spotify.com/v1/audio-analysis/33e9...,215627,4
39exKIvycQDgs4T6uXdyu0,Harmony Hall,Vampire Weekend,60,0.653,0.760,11,-6.086,1,0.0479,0.4750,0.000129,0.0917,0.600,98.008,audio_features,spotify:track:39exKIvycQDgs4T6uXdyu0,https://api.spotify.com/v1/tracks/39exKIvycQDg...,https://api.spotify.com/v1/audio-analysis/39ex...,308733,4
54YE9pzrjZToZSneNAXV99,Cardinal,Mt. Joy,49,0.544,0.296,10,-9.424,1,0.0330,0.8400,0.000091,0.1000,0.389,158.253,audio_features,spotify:track:54YE9pzrjZToZSneNAXV99,https://api.spotify.com/v1/tracks/54YE9pzrjZTo...,https://api.spotify.com/v1/audio-analysis/54YE...,209899,4
3D5RkmZmTOikPlwBVMApoW,Be Who You Are - Acoustic,The Kooks,0,0.595,0.756,8,-6.014,0,0.0362,0.1150,0.000004,0.1870,0.429,138.090,audio_features,spotify:track:3D5RkmZmTOikPlwBVMApoW,https://api.spotify.com/v1/tracks/3D5RkmZmTOik...,https://api.spotify.com/v1/audio-analysis/3D5R...,203211,4
6CYEYAIgyxmjmsnn5b2TdT,Great Ocean Road,Naya,23,0.632,0.722,8,-8.357,1,0.0406,0.0373,0.000057,0.1380,0.334,122.982,audio_features,spotify:track:6CYEYAIgyxmjmsnn5b2TdT,https://api.spotify.com/v1/tracks/6CYEYAIgyxmj...,https://api.spotify.com/v1/audio-analysis/6CYE...,201240,4
2p56oIwLWTo2qh3c3SAZMk,Bet On Me,Walk Off the Earth,58,0.690,0.537,11,-9.727,1,0.2250,0.0724,0.000000,0.3270,0.854,76.065,audio_features,spotify:track:2p56oIwLWTo2qh3c3SAZMk,https://api.spotify.com/v1/tracks/2p56oIwLWTo2...,https://api.spotify.com/v1/audio-analysis/2p56...,172984,4
68Tfpa7ydcs574ytcsIiGS,I Can't Get to Sleep,Alexander Wolfe,9,0.553,0.563,5,-7.663,0,0.0312,0.0930,0.000000,0.1100,0.382,105.035,audio_features,spotify:track:68Tfpa7ydcs574ytcsIiGS,https://api.spotify.com/v1/tracks/68Tfpa7ydcs5...,https://api.spotify.com/v1/audio-analysis/68Tf...,178528,4
5s27e14VgRZrhZsyhI0eNw,The Levee,Kingdom Jasmine,31,0.536,0.458,6,-7.613,1,0.0490,0.7020,0.000006,0.1300,0.637,157.933,audio_features,spotify:track:5s27e14VgRZrhZsyhI0eNw,https://api.spotify.com/v1/tracks/5s27e14VgRZr...,https://api.spotify.com/v1/audio-analysis/5s27...,263480,4


In [25]:
columns = ['id', 'track_name', 'artist_name', 'popularity', 'duration_ms',
        'danceability', 'time_signature', 'energy', 'key', 'loudness', 'mode', 'speechiness', 'acousticness',
        'instrumentalness', 'liveness', 'valence', 'tempo']

nominal_cols = ['id', 'track_name', 'artist_name']
discret_cols = ['key', 'mode']
continuous_cols = ['popularity', 'duration_ms', 'danceability', 'time_signature', 'loudness', 'speechiness',
        'instrumentalness', 'liveness', 'valence', 'tempo', 'acousticness*energy'] 
useful_cols = discret_cols + continuous_cols

In [26]:
final_df['acousticness*energy']=final_df['acousticness']*final_df['energy']

In [27]:
final_df=final_df[useful_cols]

In [28]:
# from sklearn import metrics
# def cv_silhouette_scorer(estimator, X):
#     estimator.fit(X)
#     cluster_labels = estimator['model'].labels_
#     num_labels = len(set(cluster_labels))
#     num_samples = len(X.index)
#     if num_labels == 1 or num_labels == num_samples:
#         return -1
#     else:
#         return metrics.silhouette_score(X, cluster_labels)

In [29]:
import pickle
model = pickle.load(open('fullkmeans1474+15.sav', 'rb'))

In [30]:
model

Pipeline(steps=[('preprocess',
                 ColumnTransformer(transformers=[('number',
                                                  Pipeline(steps=[('scale',
                                                                   MinMaxScaler())]),
                                                  ['key', 'mode', 'popularity',
                                                   'duration_ms',
                                                   'danceability',
                                                   'time_signature', 'loudness',
                                                   'speechiness',
                                                   'instrumentalness',
                                                   'liveness', 'valence',
                                                   'tempo',
                                                   'acousticness*energy'])])),
                ('model', KMeans(n_clusters=15, n_init=5))])

In [31]:
final_df.shape

(661, 13)

In [32]:
import numpy as np

In [33]:
df['prediction']=model.predict(final_df)

In [34]:
df

,track_name,artist_name,track_id,popularity,prediction
0,Postcards,James Blunt,64tit6tIpq48NSqTMzRf2d,0,1
1,Washing Dishes,Jack Johnson,22RxilGS5NSRe2ELlyKeTi,61,12
2,You're Beautiful,James Blunt,0vg4WnUWvze6pBOJDTq99k,75,3
3,Bad Day,Daniel Powter,0mUyMawtxj1CJ76kn9gIZK,75,6
4,I'm Yours,Jason Mraz,1EzrEOXmMH3G43AXT1y7pA,80,12
5,"Free Fallin' - Live at the Nokia Theatre, Los ...",John Mayer,4LloVtxNZpeh7q7xdi1DQc,71,10
6,Banana Pancakes,Jack Johnson,0BgbobvykXxEvxo2HhCuvM,69,12
7,"Sitting, Waiting, Wishing",Jack Johnson,1dnXuEuApvuVqQCn0v9McL,67,6
8,Orange Sky,Alexi Murdoch,5Szn0p3DlnawyuhX8ULCg4,0,9
9,All My Days,Alexi Murdoch,3enNOrgPG8VH9WEMzr6yPC,0,8


In [35]:
len(model.predict(final_df))

661

In [36]:
df['prediction'].value_counts()

9     130
4     130
6      79
13     65
1      54
3      44
2      36
12     32
11     29
0      22
10     12
14     11
8       8
7       5
5       4
Name: prediction, dtype: int64

In [37]:
df[['prediction', 'track_name', 'artist_name']].sort_values(by='artist_name')

,prediction,track_name,artist_name
391,9,Plus tard,47ter
346,1,Club of Optimistic People,77 Bombay Street
108,11,47 Millionaires,77 Bombay Street
469,12,That's Life (feat. Mac Miller & Sia),88-Keys
128,6,Six Years,A Blaze of Feather
457,10,Shorty Don't Wait,A Great Big World
381,4,Three Weeks,A Tale of Golden Keys
459,1,Breakin' Ground,AHI
119,9,Ol' Sweet Day,AHI
131,9,Making All Things New,Aaron Espe


Faire des paliers de suivant si on aime ou pas : + de 50%, de 50% à 5%, de 5% à 1%, de 1% à 0+%, et 0%    
Ptet faire entrer en compte la taille de la playlist pour ponderer (pas forcément necessaire)

In [51]:
len(df['prediction'])

661

In [52]:
thresh = 0.05
liked_clusters = df['prediction'].value_counts()[df['prediction'].value_counts() >= thresh*len(df['prediction'])].index

In [53]:
4 in liked_clusters

True

In [54]:
liked_clusters

Int64Index([9, 4, 6, 13, 1, 3, 2], dtype='int64')

In [55]:
df['prediction'].values[0]

1